In [129]:
pip install plotly

Note: you may need to restart the kernel to use updated packages.


In [1]:
import numpy as np

Determine computer's underflow and overflow. Underflow occurs when result = 0 and overflow when result = inf

In [2]:
def FindUnderOverFlow(N=1000):
    under = 1.0
    over = 1.0
    bool_under = False
    bool_over = False
    
    underflow_value = 0.0
    overflow_value = 0.0
    for i in range(N):
        under = under/2
        over = over*2
        if under == 0:
            bool_under = True
        else:
            underflow_value = under
        
        if over == np.inf:
            bool_over = True
        else:
            overflow_value = over
        
        if(bool_over and bool_under):
            break
    return(bool_over and bool_under, underflow_value, overflow_value)
        
print(FindUnderOverFlow(N=2000))

(True, 5e-324, 8.98846567431158e+307)


Determine computer precision (Machine Epsilon $\epsilon_m$) The $\epsilon_m$ is the smallest number that the machine recognizes as being very much greater than 0

In [3]:
def machineEpsilon(N=100, func=float):
    eps = func(1)
    bool_eps = False
    int_iter = 0
    for i in range(N):
        eps = eps/2
        one = 1.0 + eps
        if(one == 1.0):
            bool_eps = True
            int_iter = i
            break
    return bool_eps, eps*2, int_iter+1
bool_found, machine_eps, iters = machineEpsilon(func=float)
print("Bool found: ", bool_found, "\n", "Machine Epsilon: ", machine_eps, "\n",
     "Number of Iterations: ", iters)

Bool found:  True 
 Machine Epsilon:  2.220446049250313e-16 
 Number of Iterations:  53


The Dwarf number is the smalles number not equat to 0 that he machine recognises

In [4]:
def dwarfNumber(N=100, func=complex):
    eps = func(1)
    bool_found = False
    dwarf = 1
    iter_value = 0
    for i in range(N):
        eps = eps/2
        if eps == 0:
            bool_found = True
            iter_value = i
            break
        dwarf = eps
    return bool_found, dwarf, iter_value
dwarfNumber(N=1500,func=float)

(True, 5e-324, 1074)

### Numerical Derivative

Forward Difference algorithm:
\begin{equation}
\frac{dy}{dx}\bigg\rvert_{FD} \approxeq
    \frac{y(t+h) - y(t)}{h} + \mathcal{O}(h)
\end{equation}

In [5]:
def funcToDifferentiate(x=1):
    return 1

def funcEtoX(x=1, a=np.e):
    return np.power(a,x)

def fdDifferential(func=funcToDifferentiate, t = 1, h = 0.0000001):
    dfdx = (func(x=t+h) - func(x=t))/h
    return dfdx

fdDifferential(func=funcEtoX, t=1)

2.7182819639648415

Central Difference algorithm:
\begin{equation}
\frac{dy}{dx}\bigg\rvert_{FD} \approxeq
    \frac{y(t+\frac{h}{2}) - y(t-\frac{h}{2})}{h} + \mathcal{O}(h^2)
\end{equation}

In [6]:
def cdDifferential(func=funcToDifferentiate, t=1, h=0.0000001):
    dfdx = (func(x=t+h/2) - func(x=t-h/2))/h
    return dfdx

cdDifferential(funcEtoX, t=1)

2.7182818262971864

When h gets too small, like at $\epsilon_m$ roundoff error occurs and the values are incorrect

In [7]:
fdDifferential(t=1,h=machine_eps), cdDifferential(t=1, h=machine_eps)

(0.0, 0.0)

In [8]:
machine_eps

2.220446049250313e-16

The Second Derivative
\begin{equation}
\frac{d^2y}{dt^2}\bigg\rvert_{CD} \approxeq
    \frac{y(t+h) + y(t-h) - 2y(t)}{h^2}
\end{equation}

In [9]:
def secondDerivative(func=funcToDifferentiate, t=1.0, h =0.0001):
    dydt = (func(x=t+h) + func(x=t-h) - 2*func(x=t))/h**2
    return dydt

In [10]:
secondDerivative(funcEtoX, t=1.0, h=0.0001)

2.7182818662652153

In [11]:
def funcCosX(x=1):
    return np.cos(x)

In [12]:
secondDerivative(funcCosX, t = 1.0, h=0.0001)

-0.5403023362049453

In [13]:
cdDifferential(funcCosX, t=1.0, h=0.0001)

-0.8414709844584145

### Numerical Integration

\begin{equation}
\int_{a}^{b} f(x) \approxeq \sum_{i=1}^{N} f(x_i)w_i
\end{equation}

\begin{equation}
x_i = a + ih
\end{equation}

\begin{equation}
w_i = \bigg \{\frac{h}{2}, h, h, ..., h, \frac{h}{2} \bigg\} \ (Trapezoid \ Rule)
\end{equation}

\begin{equation}
w_i = \bigg \{\frac{h}{3}, \frac{4h}{3}, \frac{2h}{3}, \frac{4h}{3} ..., \frac{2h}{3}, \frac{4h}{3} , \frac{h}{3} \bigg\} \ (Simpson \ Rule) \\
\end{equation}

\begin{equation}
h = \frac{b-a}{N-1}
\end{equation}

In [14]:
def numIntegrationTrap(func=funcEtoX, N=10, a=0, b=1):
    h = (b-a)/(N-1)
    w_i = h*np.ones(N)
    w_i[0] = w_i[0]/2
    w_i[-1] = w_i[-1]/2
    int_sum = 0
    for i in range(N):
        x_i = a + i*h
        int_sum += w_i[i]*func(x=x_i)
    return int_sum

In [61]:
def numIntegrationSimp(func=funcEtoX, N=10, a=0, b=1):
    h = (b-a)/(N-1)
    w_i = h/3*np.ones(N+1)
    
    for i in range(N):
        if (i==0) or (i==N):
            w_i[i] = w_i[i]
        else:
            if((i+1)%2 == 0):
                w_i[i] = 4*w_i[i]
            else:
                w_i[i] = 2*w_i[i]

    int_sum = 0
    for i in range(N+1):
        x_i = a + i*h
        int_sum += w_i[i]*func(x=x_i)
    return int_sum

In [62]:
numIntegrationTrap(func=funcEtoX, N=100), numIntegrationSimp(func=funcEtoX, N=100)

(1.7182964381834487, 1.7458783625609586)

In [63]:
true_val = np.e - 1
numIntegrationTrap(), numIntegrationSimp(), true_val

(1.7200492444841697, 2.0377335004431476, 1.718281828459045)

#### Gaussian Quadrature (Numerical Integration)

\begin{equation}
\int_a^b f(x) dx = \int_{-1}^{1} f(t) dt = \sum_{i=1}^n A_i f(t_i) \\
f(t) = \frac{f(x)(b-a)}{2}
\end{equation}

#### Monte Carlo Integration

\begin{equation}
\int_a^b f(x) dx = \int_a^b {dx}f(x) \approx (b-a)\frac{1}{N}\sum_{i=1}^Nf(x_i) \\
x_i=random\ point\ \in (a,b)
\end{equation}

In [64]:
def monteCarloIntegration(func=funcEtoX, N=10, a=0,b=1):
    int_sum = 0
    for i in range(N):
        int_sum += func(np.random.uniform(a,b))
    return (b-a)*int_sum/N
        

In [65]:
monteCarloIntegration(N=100)

1.7441436732122082

In [66]:
import plotly as plt

In [67]:
x = np.arange(10,10000, 100)

In [68]:
y_monte_carlo = []
y_simpson = []
y_trap = []
for i in x:
    temp_val = monteCarloIntegration(N=i)
    y_monte_carlo.append(np.abs(temp_val-(np.e-1)))
    temp_val = numIntegrationSimp(N=i)
    y_simpson.append(np.abs(temp_val-(np.e-1)))
    temp_val = numIntegrationTrap(N=i)
    y_trap.append(np.abs(temp_val-(np.e-1)))

In [70]:
import plotly.graph_objects as go

In [72]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=x, y=y_monte_carlo,
                    mode='lines',
                    name='Monte Carlo'))
fig.add_trace(go.Scatter(x=x, y=y_simpson,
                    mode='lines',
                    name='Simpson'))
fig.add_trace(go.Scatter(x=x, y=y_trap,
                    mode='lines', name='Trap'))

fig.show()

In [73]:
def funcToEvaluate(x):
    temp = np.sqrt(np.power(x,3) + 5*x)
    return np.power(np.e,temp)

Evaluate:
\begin{equation}
\int_0^1 e^{\sqrt{x^3+5x}}
\end{equation}

In [80]:
int_simpson = numIntegrationSimp(N=10000,a=0,b=1,func=funcToEvaluate)
int_monte = monteCarloIntegration(N=10000,a=0,b=1,func=funcToEvaluate)
int_trap = numIntegrationTrap(N=10000,a=0,b=1,func=funcToEvaluate)

In [81]:
int_simpson, int_monte, int_trap

(5.509420787134895, 5.505951132453259, 5.5082620635032535)

### Generate Random Numbers

#### Power Residue Method

\begin{equation}
r_{i+1} = (ar_i+c)mod\ M
\end{equation}

In [82]:
def myRandom(a=57, c=1, M=256, r1=10):
    return (a*r1+c)%M

In [117]:
N = 1000
r1 = 10
lst_my_random = [r1]
for i in range(N):
    r1 = myRandom(r1=r1)
    lst_my_random.append(r1)

In [118]:
from plotly.subplots import make_subplots

In [124]:
fig = make_subplots(rows=1, cols=2)

fig.add_trace(go.Scatter(x=np.arange(N+1), y=lst_my_random,
                    mode='markers',
                    name='Power Residue Method markers'),
             row=1, col=1
             )
fig.add_trace(go.Scatter(x=np.arange(N+1), y=lst_my_random,
                    mode='lines',
                    name='Power Residue Method lines'),
              row=1,col=2
             )
fig.show()

Finding the period of when the Power Residue algorithm repeats itself under a=57, c=1, M=256, r1=10

In [128]:
n = 0
start = 0
for i in range(1,int(len(lst_my_random)/2)):
    lst_a = lst_my_random[start:i]
    lst_b = lst_my_random[i:len(lst_a)+i]
    if(lst_a==lst_b):
        print(True, n)
        lst_a = []
        lst_b = []
        start = i
    n += 1

True 255


### Central Limit Theorem